In [56]:
from user_db_api import DataBaseConnector
import json
from random import sample
from datetime import date
import random
import re
ip_info = json.load(open("./ip_info.json", 'r', encoding='utf8'))
db = DataBaseConnector(ip_info.get("Redis"))

Successfully connected to MySQL : recipe !


In [3]:
# 鎖定 大蒜、香菜、枸杞
garlic = db.redis.zrange("大蒜",0,-1)
parsley = db.redis.zrange("香菜",0,-1)
wolfberry = db.redis.zrange("枸杞",0,-1)

In [4]:
len(garlic)

43824

In [5]:
len(parsley)

6842

In [6]:
len(wolfberry)

3201

In [15]:
def random_get(ingredient):
    recipe_list = db.redis.zrange(ingredient,0,-1)
    random_sample = sample(recipe_list,10)
    content = {}
    for each in random_sample:
        ingredient_list = db.redis.hget(each,"ingredient").split(",")
        ingredient_set = set(i.split(" ")[0] for i in ingredient_list)
        content[each] = ingredient_set
    
    return content

In [107]:
garlic_recipe = random_get("大蒜")
garlic_recipe

{'30723': {'大蒜', '奇異果', '白菜', '紅茶', '雞蛋', '麵粉'},
 '89497': {'八角', '大蒜', '牛肉', '蔥', '薑', '辣椒', '酒類'},
 '98055': {'大蒜', '沙拉', '洋蔥', '海苔', '玉米', '番茄', '菇類', '萵苣', '雞蛋', '魚', '黃瓜'},
 '106051': {'大蒜', '菇類', '蝦', '豆腐', '金針菇'},
 '29820': {'九層塔', '大蒜', '薑', '辣椒', '雞腿'},
 '32436': {'大蒜', '檸檬', '洋蔥', '番茄', '筍', '義大利麵', '蝦', '蟹', '透抽', '香菜', '鴻禧菇'},
 '17637': {'大蒜', '菇類', '辣椒', '香菜', '麵包'},
 '739': {'九層塔', '大蒜', '太白粉', '豬肉', '酒類', '雞蛋'},
 '46696': {'大蒜', '干貝', '洋蔥', '薑', '雞腿'},
 '77919': {'大蒜', '洋蔥', '魚'}}

In [59]:
def get_expire(ingredient_id):
    ing_sql = '''
    select * from recipe.ingredient where `食材ID` = '{}';
    '''.format(ingredient_id)
    db.cursor.execute(ing_sql)
    ing_info = db.cursor.fetchall()
    ing_id, ing_name, expire_date = ing_info[0]
    pattern = r"([\D]*)([\d]+)"
    ex_day_number = re.match(pattern, expire_date).group(2)  # int
    return ex_day_number

In [125]:
def insert_mysql_lack(userid,ingredient):
    recipe_list = random_get(ingredient)
    
    db.mysql = db.connect_to_mysql('recipe')
    db.cursor = db.mysql.cursor()
    db_id = db.get_db_userid(userid)
    for recipe_id in recipe_list.keys():
        today = "2020-10-{}".format(random.randint(1,31))
        recipe_list[recipe_id].discard(ingredient)
        ing_num = len(recipe_list[recipe_id])
        if ing_num <= 1:
            continue
        else:
            pass
        
        intersection = sample(recipe_list[recipe_id],random.randint(1,int(ing_num/2)))
        food_id = db.redis.hmget('general_ingredient', *intersection)
        
        
        # 每一食材id: mysql 更新
        for each_ing_id in food_id:
            ex_day_number = get_expire(each_ing_id)
          
            # 新增食材資訊寫入mysql
            sql = """
            INSERT INTO refrigerator_record (使用者ID, 食材ID, 食材重量, 食材單位, 食材存放日, 食材到期日,食材取用日)
            VALUES ('{}', '{}', '{}', '{}', '{}', '{}' + interval '{}' day,"{}");
            """.format(db_id, each_ing_id, str(random.randint(10,1000)), "g", today, today, ex_day_number,today)
            db.cursor.execute(sql)
            db.mysql.commit()

            
        # 食譜紀錄增加
        sql_my_recipe_record = """                   
        INSERT INTO my_recipe_record (使用者ID, 食譜ID, 食譜使用日期) 
        VALUES ('{}', '{}', '{}');
        """.format(db_id, recipe_id, today)
        db.cursor.execute(sql_my_recipe_record)
        db.mysql.commit()
        
        
        lack = recipe_list[recipe_id] - set(intersection)
        if lack:
            lack.add(ingredient)
            lack_food_id = db.redis.hmget('general_ingredient', *lack)
            for each_lack in lack_food_id:
                sql_lack = """
                INSERT INTO ingredient_lack_record (使用者ID, 食材ID, 食譜ID, 食譜使用日期) 
                VALUES ('{}', '{}', '{}', '{}');
                """.format(db_id, each_lack, recipe_id, today)
                db.cursor.execute(sql_lack)
                db.mysql.commit()
    
    db.close_connect_to_mysql()

In [126]:
def insert_mysql(userid,ingredient):
    recipe_list = random_get(ingredient)
    
    db.mysql = db.connect_to_mysql('recipe')
    db.cursor = db.mysql.cursor()
    db_id = db.get_db_userid(userid)
    for recipe_id in recipe_list.keys():
        today = "2020-10-{}".format(random.randint(1,31))
        recipe_list[recipe_id].discard(ingredient)
        ing_num = len(recipe_list[recipe_id])
        if ing_num <= 1:
            continue
        else:
            pass
        
        intersection = sample(recipe_list[recipe_id],random.randint(1,int(ing_num/2)))
        intersection.append(ingredient)
        food_id = db.redis.hmget('general_ingredient', *intersection)
        
        
        # 每一食材id: mysql 更新
        for each_ing_id in food_id:
            ex_day_number = get_expire(each_ing_id)
          
            # 新增食材資訊寫入mysql
            sql = """
            INSERT INTO refrigerator_record (使用者ID, 食材ID, 食材重量, 食材單位, 食材存放日, 食材到期日,食材取用日)
            VALUES ('{}', '{}', '{}', '{}', '{}', '{}' + interval '{}' day,"{}");
            """.format(db_id, each_ing_id, str(random.randint(10,1000)), "g", today, today, ex_day_number,today)
            db.cursor.execute(sql)
            db.mysql.commit()

            
        # 食譜紀錄增加
        sql_my_recipe_record = """                   
        INSERT INTO my_recipe_record (使用者ID, 食譜ID, 食譜使用日期) 
        VALUES ('{}', '{}', '{}');
        """.format(db_id, recipe_id, today)
        db.cursor.execute(sql_my_recipe_record)
        db.mysql.commit()
        
        
        lack = recipe_list[recipe_id] - set(intersection)
        if lack:
            lack_food_id = db.redis.hmget('general_ingredient', *lack)
            for each_lack in lack_food_id:
                sql_lack = """
                INSERT INTO ingredient_lack_record (使用者ID, 食材ID, 食譜ID, 食譜使用日期) 
                VALUES ('{}', '{}', '{}', '{}');
                """.format(db_id, each_lack, recipe_id, today)
                db.cursor.execute(sql_lack)
                db.mysql.commit()
    
    db.close_connect_to_mysql()

In [123]:
# 這部分製造某樣食材的缺少紀錄
userid = 'U429ec102b46a5332b32c4f1a8b3b04db'
insert_mysql_lack(userid,"枸杞")

Successfully connected to MySQL : recipe !


In [130]:
# 這部分是用作想要新增某樣食材紀錄
userid = 'U0fb00ea039a1ab786346eaf24d622597'
userid2 = 'U429ec102b46a5332b32c4f1a8b3b04db'
insert_mysql(userid2,"牛肉")

Successfully connected to MySQL : recipe !


In [127]:
db.redis.hgetall("total_user_id")

{'U0822cf0978d4161a9088d5b8d435b69f': 'Bugwang',
 'U0fb00ea039a1ab786346eaf24d622597': 'yencindy',
 'U429ec102b46a5332b32c4f1a8b3b04db': 'Ryan',
 'U51051d8f36a42aca6a507da9f7312abf': 'ting1710',
 'U5fa613ee27250c701399c54a85da589d': 'Kkjlc',
 'U7a7950da1a24c106cea9b417e32f9542': 'Janice123',
 'U93dda39a1e7d10c95b34e04ba59a0844': ' abc123',
 'Ufb7f528c36dd7cb903b05345cd4450f0': '1234567'}